## inference 대화하면서 확인해보기
- prompt template은 본인이 실험할 때 학습시킨 걸로 바꿔주세요(multi 그냥 쓰면 그대로!)
- .evn 파일에 허깅페이스와 openai token이 필요합니다. 만약 openai billing이 등록되어있지 않거나 크레딧이 없으면 오류가 뜹니다.

In [1]:
import torch
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
import torch
from dotenv import load_dotenv

load_dotenv()

test_model_name = "test_4"
MODEL = f"../finetuning/model/lora_merged/{test_model_name}"

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model.eval()
model.config.use_cache = True

/opt/ml/.cache/pypoetry/virtualenvs/finalproject-n-IakgBe-py3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:17<00:00,  5.97s/it]


In [77]:
pipe = pipeline(
    'text-generation',
    model = model,
    tokenizer = tokenizer,
    device=0,
    min_new_tokens=20,
    max_new_tokens=128,
    early_stopping=True,
    do_sample=True,
    eos_token_id=2,
    repetition_penalty=1.1,
    temperature=0.9,
    top_k=50,
    top_p=0.95,
)
local_llm = HuggingFacePipeline(pipeline=pipe)

## Vector store-backed memory
[Vector store-backed memory link](https://python.langchain.com/docs/modules/memory/how_to/vectorstore_retriever_memory)

VectorStoreRetrieverMemory는 VectorDB에 메모리를 저장하고 호출될 때마다 가장 '두드러진' 상위 K개의 문서를 쿼리합니다.

이는 상호작용의 순서를 명시적으로 추적하지 않는다는 점에서 다른 대부분의 메모리 클래스와 다릅니다.

이 경우 "문서"는 이전 대화 스니펫입니다. 이는 대화 초기에 AI가 들었던 관련 정보를 참조하는 데 유용할 수 있습니다.

### chain에서 활용

In [80]:
import json
import faiss
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})
retriever = vectorstore.as_retriever(search_kwargs=dict(k=2))
memory = VectorStoreRetrieverMemory(retriever=retriever)

template = """이전 대화와 현재 대화의 명령어를 참고하여 상황에 공감하고 친절한 응답을 생성해주세요. 응답 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.\n\n[이전 대화]\n{history}\n\n[현재 대화]\n### 명령어:\n{### 명령어}\n\n### 응답:\n"""

PROMPT = PromptTemplate(
    input_variables=["history", "### 명령어"], template=template
)

conversation_with_vectors = ConversationChain(
    llm=local_llm,
    prompt=PROMPT,
    memory=memory,
    verbose=True,
    input_key='### 명령어',
    output_key='### 응답'
)

# 대답 생성 함수
def get_response(question):
    input_dict = {'### 명령어': question}
    res = conversation_with_vectors.predict(**input_dict)
    return res

In [ ]:
res = get_response('오늘은 빨리 집가고 싶어')
print(res)